# Recommender System

# Workshop 1 : วิเคราะห์ความคล้ายของประโยค

In [5]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [6]:
a = 'beach sea beach'
b = 'Beach sea sea' # ข้อสังเกต ใช้ B ตัวใหญ่

In [8]:
text = [a, b]
cv = CountVectorizer()
count_matrix = cv.fit_transform(text)

print(cv.get_feature_names()) # รายการคำ (ข้อมูล List)
print(count_matrix.toarray()) # เมทริกซ์แสดงจำนวนคำ

['beach', 'sea']
[[2 1]
 [1 2]]


In [10]:
# เพื่อให้ดูง่ายขึ้น นำข้อมูล Matrix มาเก็บใน DataFrame
term_matrix = count_matrix.todense()
cols = cv.get_feature_names()
df = pd.DataFrame(term_matrix, columns=cols, index=['a','b'])
df

,beach,sea
a,2,1
b,1,2


In [11]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim = cosine_similarity(count_matrix)
cosine_sim

array([[1. , 0.8],
       [0.8, 1. ]])

In [12]:
a = 'beach sea beach'
b = 'Beach sea sea Beach beach beach' # ข้อสังเกต ใช้ คำซ้ำมากขึ้น
text = [a, b]
cv = CountVectorizer()
count_matrix = cv.fit_transform(text)
print(cv.get_feature_names())
print(count_matrix.toarray())

['beach', 'sea']
[[2 1]
 [4 2]]


In [13]:
term_matrix = count_matrix.todense()
cols = cv.get_feature_names()
df = pd.DataFrame(term_matrix, columns=cols, index=['a','b'])
df

,beach,sea
a,2,1
b,4,2


In [14]:
cosine_sim = cosine_similarity(count_matrix)
cosine_sim

array([[1., 1.],
       [1., 1.]])

จะเห็นว่าค่าคำตอบคล้ายกันมาก(เข้าใกล้1)

# Workshop 2 : วิเคราะห์ความคล้ายของ 4 ประโยค

In [16]:
a = 'blue skirt dress lady'
b = 'blue shirt lady'
c = 'smartphone android'
d = 'charger smartphone tablet'

text = [a, b, c, d]

cv = CountVectorizer()
count_matrix = cv.fit_transform(text)

term_matrix = count_matrix.todense()
cols = cv.get_feature_names()
df = pd.DataFrame(term_matrix, columns=cols, index=['a', 'b', 'c', 'd'])
df

,android,blue,charger,dress,lady,shirt,skirt,smartphone,tablet
a,0,1,0,1,1,0,1,0,0
b,0,1,0,0,1,1,0,0,0
c,1,0,0,0,0,0,0,1,0
d,0,0,1,0,0,0,0,1,1


In [17]:
cosine_sim = cosine_similarity(count_matrix)
cosine_sim

array([[1.        , 0.57735027, 0.        , 0.        ],
       [0.57735027, 1.        , 0.        , 0.        ],
       [0.        , 0.        , 1.        , 0.40824829],
       [0.        , 0.        , 0.40824829, 1.        ]])

+++++สิ่งนี้เป็นพื้นฐานสำคัญสำหรับการแนะนำสินค้าและบริการที่คล้ายกัน

# การใช้แนะนำสินค้าที่คล้ายกัน

In [18]:
#ตรวจสอบความคล้ายคลึงทีละคู่
ref_index = 0 # มอง A
similar_score = cosine_sim[ref_index]
similar_score.round(2)

array([1.  , 0.58, 0.  , 0.  ])

a คล้ายกับ b 0.58

In [20]:
ref_index = 2 มอง C
similar_score = cosine_sim[ref_index]
similar_score.round(2)

array([0.  , 0.  , 1.  , 0.41])

c คล้ายกับ d 0.41

In [21]:
# สมมุติว่าลูกค้ากำลังดูสินค้า c ก็ยึด c เป็นหลัก แล้วทำการเปรียบเทียบกับตัวอื่นๆ
ref_index = 2
similar_score = cosine_sim[ref_index]

# นำค่าลำดับ และ ค่าดัชนีความคล้ายคลึงมารวมกันเป็น list เพื่อใช้สำหรับอ่านค่า (Access)
similar_products = list(enumerate(similar_score)) # จับคู่อันดับโดยตัวแรกเป็น 0 1 2 3 กับ ค่า cosine_sim
similar_products

[(0, 0.0), (1, 0.0), (2, 0.9999999999999998), (3, 0.408248290463863)]

In [23]:
# ทำการจัดเรียงเอาค่า score สูงสุดไว้บนสุด
sorted_recommend = sorted(similar_products,key=lambda x:x[1], reverse=True)[1:]
sorted_recommend

[(3, 0.408248290463863), (0, 0.0), (1, 0.0)]

In [27]:
# วนรอบแนะนำสินค้า
ref_index = 3
print('Top recommend for '+ df.index[ref_index]+
     '\n--------------------------')
for i,element in enumerate(sorted_recommend):
    title = df.index[element[0]] # รัน element ที่รับค่ามาจาก sortedทีละตัว
    print('{:8} {:.3f}'.format(title, sorted_recommend[i][1]))

Top recommend for d
--------------------------
d        0.408
a        0.000
b        0.000
